# Plots of LO frequency drift transforms
Refer to:


In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
# from torchsig.signals.builders.constellation import ConstellationSignalBuilder
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
#     frequency_shift,
#     multistage_polyphase_resampler,
    torchsig_float_data_type,
    torchsig_complex_data_type
)

import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

    Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 128.
        scale (int, optional): Scale factor for normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    rng = np.random.default_rng(42)
    sample_rate = 10e6
    
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 10.0,
        snr_db_max = 10.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0, 
        transforms = [],
        target_transforms = [],
        class_list = ['tone'],
        class_distribution = [1.0],
        num_samples = 1,
        seed = 1234
    )
    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 1234
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
def local_oscillator_frequency_drift(
    data: np.ndarray,
    max_drift: float = 0.01,
    max_drift_rate: float = 0.001,
    rng: np.random.Generator = np.random.default_rng(seed=None)
) -> np.ndarray:
    """Mixes data with a frequency drifting Local Oscillator (LO), with frequency drift modeled as a bounded random walk.

    Args:
        max_drift (float): Maximum absolute frequency offset. Default 0.01.
        max_drift_rate (float): Maximum drift rate over entire data sample. Default 0.001.
        rng (np.random.Generator): Random number generator. Defaults to np.random.default_rng(seed=None).

    Returns:
        np.ndarray: Data with LO drift applied.
    
    """
    rng = rng if rng else np.random.default_rng()
    N = data.size
    
    # drift modeled as random walk
    random_walk = rng.choice([-1, 1], size = N)

    # limit rate of change to at most 1/max_drift_rate times the length of the data sample
    frequency = np.cumsum(random_walk) * max_drift_rate / np.sqrt(N)

    # reset offset if max_drift bounds reached 
    while np.argmax(np.abs(frequency) > max_drift):
        idx = np.argmax(np.abs(frequency) > max_drift)
        offset = max_drift if frequency[idx] < 0 else -max_drift
        frequency[idx:] += offset

    complex_phase = np.exp(2j * np.pi * np.cumsum(frequency))    
    data = data * complex_phase

    return frequency, data
    #return data.astype(torchsig_complex_data_type)

In [ ]:
# test functional
rng = np.random.default_rng(42)

# # test case 0 with frequency out: frequency hits 100 bound @ ~6500
# N = 10000
# tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
# data_out = local_oscillator_frequency_drift(
#     data = tone_bb_data,
#     max_drift = 100,
#     max_drift_rate = 100,
#     rng = rng
# )
# plt.plot(np.abs(data_out))


# test case 1 with frequency, data out
N = 1024
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
freq_out, data_out = local_oscillator_frequency_drift(
    data = tone_bb_data,
    max_drift = 1e-1,
    max_drift_rate = 4e-1,
    rng = rng
)
plt.plot(freq_out)

In [ ]:
unwrapped_phase = np.unwrap(np.angle(data_out))
instantaneous_frequency = np.diff(unwrapped_phase)
plt.plot(instantaneous_frequency)
df = np.diff(instantaneous_frequency) 
reset_inds = np.where(np.abs(df) > (5e-1)/2)[0]
print(reset_inds)